In [2]:
from sklearn.model_selection import train_test_split, KFold, StratifiedKFold
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from lightgbm import LGBMClassifier
from sklearn.metrics import roc_auc_score
import pandas as pd
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
import numpy as np
import keras
from keras.models import Model
from keras.layers import Input,  Dense , concatenate, Reshape, Flatten, BatchNormalization, PReLU, Dropout
from keras.models import Sequential
from keras.layers.embeddings import Embedding
from keras.layers.recurrent import LSTM
from keras.layers.wrappers import Bidirectional
import numpy as np
import tensorflow

In [3]:
df_train = pd.read_csv('train.csv')
#replace policy sales 141 and 142 with 145
df_test = pd.read_csv('test.csv')

In [4]:
df_labels = pd.get_dummies(df_train['Response'].values)

In [5]:
train_id = df_train.id.values
test_id = df_test.id.values
df_train.drop(['id'], axis=1, inplace=True)
df_test.drop(['id'], axis=1, inplace=True)

In [6]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
df_train["Gender"] = le.fit_transform(df_train["Gender"].values)
df_test["Gender"] = le.transform(df_test["Gender"].values)

le = LabelEncoder()
df_train["Region_Code"] = le.fit_transform(df_train["Region_Code"].values)
df_test["Region_Code"] = le.transform(df_test["Region_Code"].values)

le = LabelEncoder()
df_train["Vehicle_Age"] = le.fit_transform(df_train["Vehicle_Age"].values)
df_test["Vehicle_Age"] = le.transform(df_test["Vehicle_Age"].values)

le = LabelEncoder()
df_train["Vehicle_Damage"] = le.fit_transform(df_train["Vehicle_Damage"].values)
df_test["Vehicle_Damage"] = le.transform(df_test["Vehicle_Damage"].values)

le = LabelEncoder()
df_train["Policy_Sales_Channel"] = le.fit_transform(df_train["Policy_Sales_Channel"].values)
df_test["Policy_Sales_Channel"] = le.transform(df_test["Policy_Sales_Channel"].values)

In [7]:
df_train['Age_premium'] = df_train['Age']/df_train['Annual_Premium']
df_test['Age_premium'] = df_test['Age']/df_test['Annual_Premium']

In [8]:
df_train['Prev_insured_vehicle_damage'] = df_train['Previously_Insured'].astype(str) + "_" +  df_train['Vehicle_Damage'].astype(str)
df_test['Prev_insured_vehicle_damage'] = df_test['Previously_Insured'].astype(str) + "_" + df_test['Vehicle_Damage'].astype(str)

le = LabelEncoder()
df_train["Prev_insured_vehicle_damage"] = le.fit_transform(df_train["Prev_insured_vehicle_damage"].values)
df_test["Prev_insured_vehicle_damage"] = le.transform(df_test["Prev_insured_vehicle_damage"].values)

In [9]:
from sklearn.preprocessing import StandardScaler

s = StandardScaler()

df_train[['Age_premium','Age', 'Driving_License','Previously_Insured', 'Annual_Premium','Vintage']] = s.fit_transform(df_train[['Age_premium','Age', 'Driving_License','Previously_Insured', 'Annual_Premium','Vintage']].values)
df_test[['Age_premium','Age', 'Driving_License','Previously_Insured', 'Annual_Premium','Vintage']] = s.transform(df_test[['Age_premium','Age', 'Driving_License','Previously_Insured', 'Annual_Premium','Vintage']].values)

In [10]:
df_train['Age_limit'] = [1 if i<18 or i>65 else 0 for i in df_train['Age'].values]
df_test['Age_limit'] = [1 if i<18 or i>65 else 0 for i in df_test['Age'].values]

In [11]:
# Saves the predictions of provided model
def save_preds(model, epoch):
  # Create input for the model
  preds = model.predict(x = [np.reshape(df_test.Gender.values, (-1,1)),
               np.reshape(df_test.Region_Code.values, (-1,1)),
               np.reshape(df_test.Prev_insured_vehicle_damage.values, (-1,1)),
               np.reshape(df_test.Vehicle_Age.values, (-1,1)),
               np.reshape(df_test.Vehicle_Damage.values, (-1,1)),
               np.reshape(df_test.Policy_Sales_Channel.values, (-1,1)),
               df_test[['Experience','Age', 'Driving_License','Previously_Insured', 'Annual_Premium','Vintage', 'Age_premium', 'Age_limit']]], 
         verbose = 1)
  df_submit = pd.DataFrame({'id':test_id,'Response': preds[:,-1]})
  df_submit.to_csv("submit_nn_" + str(epoch) + ".csv", index=False)


class CustomCallback(tensorflow.keras.callbacks.Callback):
    def on_epoch_begin(self, epoch, logs=None):
      if epoch!=0:
        save_preds(self.model, epoch)


In [ ]:
#find average experience with respect to region code

region = df_train['Age'].values
region = np.append(region,df_test['Age'].values)

ap = df_train['Vehicle_Damage'].values
ap = np.append(ap,df_test['Vehicle_Damage'].values)

df_temp = pd.DataFrame({'Age':region, 'Vehicle_Damage':ap})

k = df_temp.groupby('Age')['Vehicle_Damage'].mean()
index = k.index.values
values = k.values

map_reg = {}

for i in range(len(index)):
  map_reg[index[i]] = values[i]

df_train['Experience'] = df_train['Age'].map(map_reg)
df_test['Experience'] = df_test['Age'].map(map_reg)

In [13]:
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
 
X_train, Y = df_train.drop(["Response"], axis=1).values, df_train["Response"].values
X_test = df_test.values
Y = df_labels.values

In [14]:
tensorflow.random.set_seed(2)
session_conf = tensorflow.compat.v1.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1)
sess = tensorflow.compat.v1.Session(graph=tensorflow.compat.v1.get_default_graph(), config=session_conf)
tensorflow.compat.v1.keras.backend.set_session(sess)

batch_size = 64

#categorical inputs
inp1 = Input(shape=(1,))
inp2 = Input(shape=(1,))
inp3 = Input(shape=(1,))
inp4 = Input(shape=(1,))
inp5 = Input(shape=(1,))
inp6 = Input(shape=(1,))


# numerical input
inp7 = Input(shape=(8,)) 
#inp7 = Input(shape=(1,))
#embedding layers
layer1 = Flatten()(Embedding(len(df_train.Gender.unique()), 2*len(df_train.Gender.unique()), trainable=True)(inp1))
layer2 = Flatten()(Embedding(len(df_train.Region_Code.unique()),2*len(df_train.Region_Code.unique()), trainable=True)(inp2))
layer3 = Flatten()(Embedding(len(df_train.Prev_insured_vehicle_damage.unique()),2*len(df_train.Prev_insured_vehicle_damage.unique()), trainable=True)(inp3))
layer4 = Flatten()(Embedding(len(df_train.Vehicle_Age.unique()),2*len(df_train.Vehicle_Age.unique()), trainable=True)(inp4))
layer5 = Flatten()(Embedding(len(df_train.Vehicle_Damage.unique()), 2*len(df_train.Vehicle_Damage.unique()), trainable=True)(inp5))
layer6 = Flatten()(Embedding(len(df_train.Policy_Sales_Channel.unique()), 2*len(df_train.Policy_Sales_Channel.unique()), trainable=True)(inp6))



merge = concatenate([layer1,layer2,layer3,layer4,layer5,layer6,inp7])

opt = keras.optimizers.Adam(learning_rate=0.0005)
hidden1 = Dense(256, activation='linear')(merge)
hidden2 = PReLU()(hidden1)

hidden1 = Dense(512, activation='linear')(hidden2)
hidden3 = PReLU()(hidden1)

hidden1 = Dense(256, activation='linear')(hidden3)
hidden1 = PReLU()(hidden1)

hidden1 = Dense(128, activation='linear')(concatenate([hidden1, hidden2]))
hidden1 = PReLU()(hidden1)

output = Dense(2,activation='softmax')(hidden1)
model = Model(inputs=[inp1,inp2,inp3,inp4,inp5,inp6,inp7], outputs=output)
model.compile(loss = 'categorical_crossentropy', optimizer=opt, metrics='mse')
print(model.summary())

model.fit(x = [np.reshape(df_train.Gender.values, (-1,1)),
               np.reshape(df_train.Region_Code.values, (-1,1)),
               np.reshape(df_train.Prev_insured_vehicle_damage.values, (-1,1)),
               np.reshape(df_train.Vehicle_Age.values, (-1,1)),
               np.reshape(df_train.Vehicle_Damage.values, (-1,1)),
               np.reshape(df_train.Policy_Sales_Channel.values, (-1,1)),
               df_train[['Experience','Age', 'Driving_License','Previously_Insured', 'Annual_Premium','Vintage', 'Age_premium', 'Age_limit']]], 
          y = Y, batch_size =batch_size, epochs = 4,  verbose = 1, callbacks=[CustomCallback()])

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 1)]          0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, 1)]          0                                            
__________________________________________________________________________________________________
input_3 (InputLayer)            [(None, 1)]          0                                            
__________________________________________________________________________________________________
input_4 (InputLayer)            [(None, 1)]          0                                            
_______________________________________________________________________________________

In [ ]:
preds = model.predict(x = [np.reshape(df_train.Gender.values, (-1,1)),
               np.reshape(df_train.Region_Code.values, (-1,1)),
               np.reshape(df_train.Prev_insured_vehicle_damage.values, (-1,1)),
               np.reshape(df_train.Vehicle_Age.values, (-1,1)),
               np.reshape(df_train.Vehicle_Damage.values, (-1,1)),
               np.reshape(df_train.Policy_Sales_Channel.values, (-1,1)),
               df_train[['Age', 'Driving_License','Previously_Insured', 'Annual_Premium','Vintage', 'Age_premium', 'Age_limit']]], 
         verbose = 1)
score = roc_auc_score(df_train['Response'].values, preds[:,-1])
print(score)

In [15]:
preds = model.predict(x = [np.reshape(df_test.Gender.values, (-1,1)),
               np.reshape(df_test.Region_Code.values, (-1,1)),
               np.reshape(df_test.Prev_insured_vehicle_damage.values, (-1,1)),
               np.reshape(df_test.Vehicle_Age.values, (-1,1)),
               np.reshape(df_test.Vehicle_Damage.values, (-1,1)),
               np.reshape(df_test.Policy_Sales_Channel.values, (-1,1)),
               df_test[['Experience','Age', 'Driving_License','Previously_Insured', 'Annual_Premium','Vintage', 'Age_premium', 'Age_limit']]], 
         verbose = 1)
df_submit = pd.DataFrame({'id':test_id,'Response': preds[:,-1]})
df_submit.to_csv("submit_nn.csv", index=False)

3970/3970 [==============================] - 8s 2ms/step
